## Importe de bibliotecas

In [2]:
import multiprocessing

from gensim.models import Word2Vec

import pandas as pd

## Carga de datos

In [3]:
resumenes_docentes = pd.read_csv("../ResumenesDocentes.csv")
resumenes_estudiantes = pd.read_csv("../ResumenesEstudiantes.csv")

## Verificacion datos

In [ ]:
resumenes_docentes.shape

In [ ]:
resumenes_docentes.head(3)

In [ ]:
resumenes_estudiantes.shape

In [ ]:
resumenes_estudiantes.head(3)

## Bigramas

In [ ]:
from gensim.models.phrases import Phrases, Phraser

In [ ]:
phrases = Phrases(lemas_fila1_spliteado, min_count=30, progress_per=10000)

In [ ]:
bigram = Phraser(phrases)

In [ ]:
sentences = bigram[lemas_fila1_spliteado]

## Preparación de datos

In [4]:
lemas_corpus_docentes=resumenes_docentes['lemas_corpus']

In [ ]:
# lemas_fila1=lemas_corpus_docentes[0]

In [ ]:
# lemas_fila1

In [ ]:
# lemas_fila1_spliteado=lemas_fila1.split()

In [ ]:
# len(lemas_fila1_spliteado)

In [111]:
lemas_corpus_docentes_agrupados = ""
for row in lemas_corpus_docentes:
    lemas_corpus_docentes_agrupados  = lemas_corpus_docentes_agrupados  + row

In [112]:
lemas_corpus_docentes_spliteados = lemas_corpus_docentes_agrupados .split()

In [113]:
lemas_corpus_docentes_spliteados

['disponer',
 'profesional',
 'identificar',
 'papel',
 'beneficio',
 'flexible',
 'pretender',
 'conveniente',
 'permitir',
 'aporte',
 'basico',
 'relacionar',
 'formador',
 'estudio',
 'gradar',
 'nivel',
 'complejidad',
 'grupo',
 'continuo',
 'fomentar',
 'tecnologico',
 'publico',
 'beneficiar',
 'colectivo',
 'creativo',
 'afrontar',
 'emprendedor',
 'promover',
 'experiencia',
 'academico',
 'proponer',
 'cambiar',
 'universidad',
 'individual',
 'academico',
 'evolucionar',
 'procesar',
 'ayala',
 'complejo',
 'interesar',
 'tipificar',
 'expectativa',
 'escuchar',
 'importancia',
 'modalidad',
 'participar',
 'asociar',
 'dinamico',
 'egresar',
 'sostenible',
 'vital',
 'plantear',
 'particular',
 'catalogar',
 'individuo',
 'aportar',
 'estimulante',
 'concienciar',
 'principio',
 'desarrollo',
 'humano',
 'conjuntar',
 'capaz',
 'capacidad',
 'social',
 'industriar',
 'extremar',
 'individuo',
 'crecimiento',
 'innovador',
 'asumir',
 'diseñar',
 'estrategico',
 'creacion',

## Word2vec

In [106]:
# Fase de creación de modelo

modelo = Word2Vec(min_count=1,
                     window=1,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=7)
modelo

In [107]:
# Fase de construcción de vocabulario

modelo.build_vocab([lemas_corpus_docentes_spliteados], progress_per=10000)

In [108]:
# Fase de entrenamiento

modelo.train([lemas_corpus_docentes_spliteados], total_examples=modelo.corpus_count, epochs=30, report_delay=1)

(300000, 1274610)

In [ ]:
# para no volver a entrenar
# w2v_model.init_sims(replace=True)

## Exploración de modelo

In [109]:
modelo.wv.most_similar(positive=['motor'])

[('fisico', 0.9999262690544128),
 ('contribuir', 0.9999228119850159),
 ('agente', 0.9999215006828308),
 ('tratamiento', 0.999921441078186),
 ('mejoramiento', 0.9999213814735413),
 ('vegetal', 0.9999210834503174),
 ('cantidad', 0.9999210238456726),
 ('diseñar', 0.9999207854270935),
 ('elevar', 0.9999204874038696),
 ('animal', 0.9999198317527771)]

In [110]:
modelo.wv.similarity('social', "sociedad")

0.9999498